In [13]:
from headers import *

In [14]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

In [15]:
df = pd.read_excel("input_df.xlsx")

In [16]:
#df.drop(df.columns[-1],axis = 1, inplace = True)
df.index.name = "index"

In [17]:
df.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
0,1,1,1,0,20,0,20,10,0
1,1,2,1,0,20,0,20,5,1
2,1,3,1,20,0,20,0,10,0
3,1,4,1,20,0,20,0,5,1
4,1,5,1,20,0,0,20,10,0
5,1,6,1,20,0,0,20,5,1
6,2,1,1,0,15,0,15,10,0
7,2,2,1,0,15,0,15,5,0
8,2,3,1,15,0,15,0,10,0


# Model 01 - all trips: 
### dep1 = [ASC_now] + ivt_4 * [B_ivt4_now] + ivt_3 * [B_ivt3_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt3_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [18]:
# filter records
#df_used = df[(df['time_crit']==0) & (df['scenario'] > 4) & (df['scenario'] < 7)]
df_used = df

In [19]:
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
0,1,1,1,0,20,0,20,10,0
1,1,2,1,0,20,0,20,5,1
2,1,3,1,20,0,20,0,10,0
3,1,4,1,20,0,20,0,5,1
4,1,5,1,20,0,0,20,10,0
5,1,6,1,20,0,0,20,5,1
6,2,1,1,0,15,0,15,10,0
7,2,2,1,0,15,0,15,5,0
8,2,3,1,15,0,15,0,10,0


In [20]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [21]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
NOW_IVT_3 = now_ivt_3
WAIT_IVT_3 = wait_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4 + \
     B_IVT_3_now * NOW_IVT_3

V2 = ASC_WAIT + \
     B_IVT_3_wait * WAIT_IVT_3 + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [22]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_01_EDI_all_trips_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	1.09
BETA_IVT_2_WAIT=	0.00628
BETA_IVT_3_NOW=	0.0289
BETA_IVT_3_WAIT=	0.00285
BETA_IVT_4_NOW=	-0.0381
BETA_WT_WAIT=	-0.236
                    Value   Std err     t-test       p-value  Rob. Std err  \
ASC_WAIT         1.094410  0.167954   6.516139  7.213985e-11      0.173918   
BETA_IVT_2_WAIT  0.006280  0.002862   2.194274  2.821575e-02      0.003367   
BETA_IVT_3_NOW   0.028944  0.003877   7.466533  8.237855e-14      0.004424   
BETA_IVT_3_WAIT  0.002846  0.003658   0.777903  4.366260e-01      0.004277   
BETA_IVT_4_NOW  -0.038070  0.002890 -13.175312  0.000000e+00      0.003425   
BETA_WT_WAIT    -0.235655  0.018883 -12.479478  0.000000e+00      0.018750   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            6.292667  3.120568e-10  
BETA_IVT_2_WAIT     1.865242  6.214744e-02  
BETA_IVT_3_NOW      6.542777  6.038703e-11  
BETA_IVT_3_WAIT     0.665272  5.058766e-01  
BETA_IVT_4_NOW    -11.116294  0.000000e+00  
BETA_WT_WAIT      -12.568263  0.000000e+00  


# Model 02 - all trips, no ASC: 
### dep1 = ivt_4 * [B_ivt4_now] + ivt_3 * [B_ivt3_now] 
### dep2 = ivt_3 * [B_ivt3_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [23]:
# coefficients
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
NOW_IVT_3 = now_ivt_3
WAIT_IVT_3 = wait_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = B_IVT_4_now * NOW_IVT_4 + \
     B_IVT_3_now * NOW_IVT_3

V2 = B_IVT_3_wait * WAIT_IVT_3 + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [24]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_02_EDI_all_trips"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_2_WAIT=	0.0137
BETA_IVT_3_NOW=	0.0203
BETA_IVT_3_WAIT=	0.0102
BETA_IVT_4_NOW=	-0.0442
BETA_WT_WAIT=	-0.139
                    Value   Std err     t-test       p-value  Rob. Std err  \
BETA_IVT_2_WAIT  0.013665  0.002615   5.225877  1.733312e-07      0.003017   
BETA_IVT_3_NOW   0.020328  0.003493   5.819512  5.901961e-09      0.003807   
BETA_IVT_3_WAIT  0.010197  0.003476   2.933227  3.354586e-03      0.004080   
BETA_IVT_4_NOW  -0.044190  0.002768 -15.964569  0.000000e+00      0.003282   
BETA_WT_WAIT    -0.139189  0.011446 -12.160119  0.000000e+00      0.012423   

                 Rob. t-test  Rob. p-value  
BETA_IVT_2_WAIT     4.529990  5.898649e-06  
BETA_IVT_3_NOW      5.339782  9.305821e-08  
BETA_IVT_3_WAIT     2.499050  1.245268e-02  
BETA_IVT_4_NOW    -13.465190  0.000000e+00  
BETA_WT_WAIT      -11.203970  0.000000e+00  


# Model 03 - non-time-critical trips only: 
### dep1 = [ASC_now] + ivt_4 * [B_ivt4_now] + ivt_3 * [B_ivt3_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt3_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [25]:
# filter records
#df_used = df[(df['time_crit']==0) & (df['scenario'] > 4) & (df['scenario'] < 7)]
df_used = df[df['time_crit'] == 0]
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
24,5,1,0,0,20,0,20,10,1
25,5,2,0,0,20,0,20,5,1
26,5,3,0,20,0,20,0,10,1
27,5,4,0,20,0,20,0,5,1
28,5,5,0,20,0,0,20,10,1
29,5,6,0,20,0,0,20,5,1
36,7,1,0,0,10,0,10,10,1
37,7,2,0,0,10,0,10,5,1
38,7,3,0,10,0,10,0,10,1


In [26]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [27]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
NOW_IVT_3 = now_ivt_3
WAIT_IVT_3 = wait_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4 + \
     B_IVT_3_now * NOW_IVT_3

V2 = ASC_WAIT + \
     B_IVT_3_wait * WAIT_IVT_3 + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [28]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_03_EDI_nontimecrit_trips_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	1.44
BETA_IVT_2_WAIT=	0.0151
BETA_IVT_3_NOW=	0.03
BETA_IVT_3_WAIT=	0.0097
BETA_IVT_4_NOW=	-0.0548
BETA_WT_WAIT=	-0.248
                    Value   Std err     t-test       p-value  Rob. Std err  \
ASC_WAIT         1.442851  0.246949   5.842700  5.136134e-09      0.263754   
BETA_IVT_2_WAIT  0.015142  0.004530   3.342380  8.306316e-04      0.005668   
BETA_IVT_3_NOW   0.029982  0.005352   5.601843  2.120846e-08      0.006032   
BETA_IVT_3_WAIT  0.009702  0.005846   1.659717  9.697146e-02      0.007441   
BETA_IVT_4_NOW  -0.054826  0.004848 -11.308283  0.000000e+00      0.006644   
BETA_WT_WAIT    -0.247530  0.026814  -9.231234  0.000000e+00      0.026243   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            5.470444  4.489107e-08  
BETA_IVT_2_WAIT     2.671686  7.547127e-03  
BETA_IVT_3_NOW      4.970462  6.679346e-07  
BETA_IVT_3_WAIT     1.303849  1.922850e-01  
BETA_IVT_4_NOW     -8.252388  2.220446e-16  
BETA_WT_WAIT       -9.432100  0.000000e+00  


# Model 04 - non-time-critical trips only, no ASC: 
### dep1 = ivt_4 * [B_ivt4_now] + ivt_3 * [B_ivt3_now] 
### dep2 = ivt_3 * [B_ivt3_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [29]:
# coefficients
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
NOW_IVT_3 = now_ivt_3
WAIT_IVT_3 = wait_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = B_IVT_4_now * NOW_IVT_4 + \
     B_IVT_3_now * NOW_IVT_3

V2 = B_IVT_3_wait * WAIT_IVT_3 + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [30]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_04_EDI_nontimecrit_trips_no_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_2_WAIT=	0.0256
BETA_IVT_3_NOW=	0.0191
BETA_IVT_3_WAIT=	0.02
BETA_IVT_4_NOW=	-0.0646
BETA_WT_WAIT=	-0.125
                    Value   Std err     t-test       p-value  Rob. Std err  \
BETA_IVT_2_WAIT  0.025592  0.004189   6.109047  1.002281e-09      0.005104   
BETA_IVT_3_NOW   0.019056  0.004858   3.922303  8.770647e-05      0.005430   
BETA_IVT_3_WAIT  0.019952  0.005638   3.538627  4.022135e-04      0.007188   
BETA_IVT_4_NOW  -0.064599  0.004622 -13.976314  0.000000e+00      0.006124   
BETA_WT_WAIT    -0.124756  0.016220  -7.691696  1.443290e-14      0.018856   

                 Rob. t-test  Rob. p-value  
BETA_IVT_2_WAIT     5.014031  5.330149e-07  
BETA_IVT_3_NOW      3.509621  4.487456e-04  
BETA_IVT_3_WAIT     2.775797  5.506651e-03  
BETA_IVT_4_NOW    -10.549374  0.000000e+00  
BETA_WT_WAIT       -6.616346  3.681877e-11  


# Model 05 - time-critical trips only: 
### dep1 = [ASC_now] + ivt_4 * [B_ivt4_now] + ivt_3 * [B_ivt3_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt3_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [31]:
# filter records
#df_used = df[(df['time_crit']==0) & (df['scenario'] > 4) & (df['scenario'] < 7)]
df_used = df[df['time_crit'] == 1]
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
0,1,1,1,0,20,0,20,10,0
1,1,2,1,0,20,0,20,5,1
2,1,3,1,20,0,20,0,10,0
3,1,4,1,20,0,20,0,5,1
4,1,5,1,20,0,0,20,10,0
5,1,6,1,20,0,0,20,5,1
6,2,1,1,0,15,0,15,10,0
7,2,2,1,0,15,0,15,5,0
8,2,3,1,15,0,15,0,10,0


In [32]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [33]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
NOW_IVT_3 = now_ivt_3
WAIT_IVT_3 = wait_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4 + \
     B_IVT_3_now * NOW_IVT_3

V2 = ASC_WAIT + \
     B_IVT_3_wait * WAIT_IVT_3 + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [34]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_05_EDI_timecrit_trips_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	0.811
BETA_IVT_2_WAIT=	0.00291
BETA_IVT_3_NOW=	0.0309
BETA_IVT_3_WAIT=	9.61e-05
BETA_IVT_4_NOW=	-0.034
BETA_WT_WAIT=	-0.291
                    Value   Std err    t-test   p-value  Rob. Std err  \
ASC_WAIT         0.811388  0.255325  3.177864  0.001484      0.257343   
BETA_IVT_2_WAIT  0.002914  0.004218  0.690718  0.489743      0.005097   
BETA_IVT_3_NOW   0.030941  0.006694  4.622041  0.000004      0.008690   
BETA_IVT_3_WAIT  0.000096  0.005305  0.018113  0.985549      0.006043   
BETA_IVT_4_NOW  -0.033951  0.004023 -8.439790  0.000000      0.004092   
BETA_WT_WAIT    -0.290874  0.031502 -9.233421  0.000000      0.031684   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            3.152950      0.001616  
BETA_IVT_2_WAIT     0.571571      0.567613  
BETA_IVT_3_NOW      3.560687      0.000370  
BETA_IVT_3_WAIT     0.015903      0.987312  
BETA_IVT_4_NOW     -8.297097      0.000000  
BETA_WT_WAIT       -9.180452      0.000000  


# Model 06 - time-critical trips only, no ASC: 
### dep1 = ivt_4 * [B_ivt4_now] + ivt_3 * [B_ivt3_now] 
### dep2 = ivt_3 * [B_ivt3_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [35]:
# coefficients
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
NOW_IVT_3 = now_ivt_3
WAIT_IVT_3 = wait_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = B_IVT_4_now * NOW_IVT_4 + \
     B_IVT_3_now * NOW_IVT_3

V2 = B_IVT_3_wait * WAIT_IVT_3 + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [36]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_06_EDI_timecrit_trips_no_ASC"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_IVT_2_WAIT=	0.0079
BETA_IVT_3_NOW=	0.0242
BETA_IVT_3_WAIT=	0.00513
BETA_IVT_4_NOW=	-0.0372
BETA_WT_WAIT=	-0.213
                    Value   Std err     t-test   p-value  Rob. Std err  \
BETA_IVT_2_WAIT  0.007897  0.003857   2.047225  0.040636      0.004511   
BETA_IVT_3_NOW   0.024158  0.006093   3.965193  0.000073      0.007416   
BETA_IVT_3_WAIT  0.005129  0.004997   1.026343  0.304730      0.005571   
BETA_IVT_4_NOW  -0.037184  0.003893  -9.550906  0.000000      0.004040   
BETA_WT_WAIT    -0.212854  0.018930 -11.244073  0.000000      0.020357   

                 Rob. t-test  Rob. p-value  
BETA_IVT_2_WAIT     1.750583      0.080018  
BETA_IVT_3_NOW      3.257429      0.001124  
BETA_IVT_3_WAIT     0.920680      0.357218  
BETA_IVT_4_NOW     -9.204026      0.000000  
BETA_WT_WAIT      -10.455841      0.000000  


## Models 07 - 12: separate models for each experimental set, i.e:
### experiment no. [1]: 1st dep [***], 2nd dep [**]
### experiment no. [2]: 1st dep [****], 2nd dep [***]
### experiment no. [3]: 1st dep [****], 2nd dep [**]

# Model 07 - experiment no. 1, non-time-critical trips only: 
### dep1 = [ASC_now] + ivt_3 * [B_ivt3_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [68]:
# filter records
df_used = df[(df['time_crit']==0) & ((df['scenario'] == 1) | (df['scenario'] == 2))]
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
24,5,1,0,0,20,0,20,10,1
25,5,2,0,0,20,0,20,5,1
36,7,1,0,0,10,0,10,10,1
37,7,2,0,0,10,0,10,5,1
54,10,1,0,0,25,0,25,10,0
55,10,2,0,0,25,0,25,5,1
60,11,1,0,0,17,0,17,10,0
61,11,2,0,0,17,0,17,5,0
66,12,1,0,0,5,0,5,10,0


In [69]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [70]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_3 = now_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_3_now * NOW_IVT_3

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [71]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_07_EDI_nontimecrit_trips_ASC_scenario_1"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	0.17
BETA_IVT_2_WAIT=	0.0145
BETA_IVT_3_NOW=	-0.0145
BETA_WT_WAIT=	-0.237
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         0.169961  0.398432  0.426575  6.696893e-01      0.411924   
BETA_IVT_2_WAIT  0.014550  0.005174  2.812063  4.922489e-03      0.005692   
BETA_IVT_3_NOW  -0.014550  0.005174 -2.812063  4.922489e-03      0.005692   
BETA_WT_WAIT    -0.236733  0.046489 -5.092283  3.537782e-07      0.046361   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            0.412603  6.798975e-01  
BETA_IVT_2_WAIT     2.556243  1.058091e-02  
BETA_IVT_3_NOW     -2.556243  1.058091e-02  
BETA_WT_WAIT       -5.106343  3.284537e-07  


# Model 07a - experiment no. 1, non-time-critical trips only, 2nd-dep utility: 
### dep1 = [ASC_now]  
### dep2 = [ASC_wait] + ivt_2 * [B_ivt3-2_wait] + wt * [B_WT_wait]

In [72]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_3_2_wait = Beta('BETA_IVT_3_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW

V2 = ASC_WAIT + \
     B_IVT_3_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [73]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_07a_EDI_nontimecrit_trips_ASC_scenario_1_2nd_dep_only"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	0.17
BETA_IVT_3_2_WAIT=	0.0291
BETA_WT_WAIT=	-0.237
                      Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT           0.169961  0.398432  0.426574  6.696897e-01      0.411924   
BETA_IVT_3_2_WAIT  0.029099  0.010348  2.812066  4.922446e-03      0.011384   
BETA_WT_WAIT      -0.236733  0.046489 -5.092284  3.537762e-07      0.046361   

                   Rob. t-test  Rob. p-value  
ASC_WAIT              0.412602  6.798979e-01  
BETA_IVT_3_2_WAIT     2.556245  1.058085e-02  
BETA_WT_WAIT         -5.106344  3.284520e-07  


# Model 08 - experiment no. 1, time-critical trips only: 
### dep1 = [ASC_now] + ivt_3 * [B_ivt3_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [74]:
# filter records
df_used = df[(df['time_crit']==1) & ((df['scenario'] == 1) | (df['scenario'] == 2))]
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
0,1,1,1,0,20,0,20,10,0
1,1,2,1,0,20,0,20,5,1
6,2,1,1,0,15,0,15,10,0
7,2,2,1,0,15,0,15,5,0
12,3,1,1,0,15,0,15,10,0
13,3,2,1,0,15,0,15,5,0
18,4,1,1,0,30,0,30,10,0
19,4,2,1,0,30,0,30,5,0
30,6,1,1,0,25,0,25,10,0


In [75]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [76]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_3_now = Beta('BETA_IVT_3_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_3 = now_ivt_3
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_3_now * NOW_IVT_3

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [77]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_08_EDI_timecrit_trips_ASC_scenario_1"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	-0.929
BETA_IVT_2_WAIT=	0.0159
BETA_IVT_3_NOW=	-0.0159
BETA_WT_WAIT=	-0.303
                    Value   Std err    t-test   p-value  Rob. Std err  \
ASC_WAIT        -0.928565  0.644359 -1.441068  0.149565      0.614852   
BETA_IVT_2_WAIT  0.015930  0.006224  2.559598  0.010479      0.005628   
BETA_IVT_3_NOW  -0.015930  0.006224 -2.559598  0.010479      0.005628   
BETA_WT_WAIT    -0.302763  0.088890 -3.406040  0.000659      0.090150   

                 Rob. t-test  Rob. p-value  
ASC_WAIT           -1.510226      0.130986  
BETA_IVT_2_WAIT     2.830512      0.004647  
BETA_IVT_3_NOW     -2.830512      0.004647  
BETA_WT_WAIT       -3.358434      0.000784  


# Model 08a - experiment no. 1, time-critical trips only, 2nd-dep utility: 
### dep1 = [ASC_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt3-2_wait] + wt * [B_WT_wait]

In [78]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_3_2_wait = Beta('BETA_IVT_3_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW

V2 = ASC_WAIT + \
     B_IVT_3_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [79]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_08a_EDI_timecrit_trips_ASC_scenario_1_2nd_dep_only"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	-0.929
BETA_IVT_3_2_WAIT=	0.0319
BETA_WT_WAIT=	-0.303
                      Value   Std err    t-test   p-value  Rob. Std err  \
ASC_WAIT          -0.928578  0.644359 -1.441088  0.149560      0.614851   
BETA_IVT_3_2_WAIT  0.031860  0.012447  2.559613  0.010479      0.011256   
BETA_WT_WAIT      -0.302762  0.088890 -3.406031  0.000659      0.090150   

                   Rob. t-test  Rob. p-value  
ASC_WAIT             -1.510247      0.130980  
BETA_IVT_3_2_WAIT     2.830530      0.004647  
BETA_WT_WAIT         -3.358430      0.000784  


# Model 09 - experiment no. 2, non-time-critical trips only: 
### dep1 = [ASC_now] + ivt_4 * [B_ivt4_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt3_wait] + wt * [B_WT_wait]

In [80]:
# filter records
df_used = df[(df['time_crit']==0) & ((df['scenario'] == 3) | (df['scenario'] == 4))]
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
26,5,3,0,20,0,20,0,10,1
27,5,4,0,20,0,20,0,5,1
38,7,3,0,10,0,10,0,10,1
39,7,4,0,10,0,10,0,5,1
56,10,3,0,25,0,25,0,10,1
57,10,4,0,25,0,25,0,5,1
62,11,3,0,17,0,17,0,10,0
63,11,4,0,17,0,17,0,5,1
68,12,3,0,5,0,5,0,10,1


In [81]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [82]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_3 = wait_ivt_3
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = ASC_WAIT + \
     B_IVT_3_wait * WAIT_IVT_3 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [83]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_09_EDI_nontimecrit_trips_ASC_scenario_2"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	2.27
BETA_IVT_3_WAIT=	0.0175
BETA_IVT_4_NOW=	-0.0175
BETA_WT_WAIT=	-0.261
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         2.269397  0.458124  4.953677  7.282411e-07      0.506188   
BETA_IVT_3_WAIT  0.017458  0.005909  2.954660  3.130143e-03      0.008689   
BETA_IVT_4_NOW  -0.017458  0.005909 -2.954660  3.130143e-03      0.008689   
BETA_WT_WAIT    -0.261114  0.047904 -5.450752  5.015715e-08      0.047609   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            4.483309  7.349439e-06  
BETA_IVT_3_WAIT     2.009177  4.451842e-02  
BETA_IVT_4_NOW     -2.009177  4.451842e-02  
BETA_WT_WAIT       -5.484591  4.144279e-08  


# Model 09a - experiment no. 2, non-time-critical trips only, 2nd-dep utility: 
### dep1 = [ASC_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt4-3_wait] + wt * [B_WT_wait]

In [84]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_3_wait = Beta('BETA_IVT_4_3_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
WAIT_IVT_3 = wait_ivt_3
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW 

V2 = ASC_WAIT + \
     B_IVT_4_3_wait * WAIT_IVT_3 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [85]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_09a_EDI_nontimecrit_trips_ASC_scenario_2_2nd_dep_only"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	2.27
BETA_IVT_4_3_WAIT=	0.0349
BETA_WT_WAIT=	-0.261
                      Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT           2.269399  0.458124  4.953681  7.282266e-07      0.506188   
BETA_IVT_4_3_WAIT  0.034916  0.011817  2.954658  3.130156e-03      0.017378   
BETA_WT_WAIT      -0.261114  0.047904 -5.450755  5.015634e-08      0.047609   

                   Rob. t-test  Rob. p-value  
ASC_WAIT              4.483311  7.349354e-06  
BETA_IVT_4_3_WAIT     2.009175  4.451854e-02  
BETA_WT_WAIT         -5.484592  4.144244e-08  


# Model 10 - experiment no. 2, time-critical trips only: 
### dep1 = [ASC_now] + ivt_4 * [B_ivt4_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt3_wait] + wt * [B_WT_wait]

In [86]:
# filter records
df_used = df[(df['time_crit']==1) & ((df['scenario'] == 3) | (df['scenario'] == 4))]
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
2,1,3,1,20,0,20,0,10,0
3,1,4,1,20,0,20,0,5,1
8,2,3,1,15,0,15,0,10,0
9,2,4,1,15,0,15,0,5,0
14,3,3,1,15,0,15,0,10,0
15,3,4,1,15,0,15,0,5,1
20,4,3,1,30,0,30,0,10,0
21,4,4,1,30,0,30,0,5,1
32,6,3,1,25,0,25,0,10,1


In [87]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [88]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_3_wait = Beta('BETA_IVT_3_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_3 = wait_ivt_3
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = ASC_WAIT + \
     B_IVT_3_wait * WAIT_IVT_3 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [89]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_10_EDI_timecrit_trips_ASC_scenario_2"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	1.25
BETA_IVT_3_WAIT=	0.00905
BETA_IVT_4_NOW=	-0.00905
BETA_WT_WAIT=	-0.286
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         1.251089  0.401908  3.112876  1.852740e-03      0.398442   
BETA_IVT_3_WAIT  0.009049  0.004558  1.985551  4.708320e-02      0.004468   
BETA_IVT_4_NOW  -0.009049  0.004558 -1.985551  4.708320e-02      0.004468   
BETA_WT_WAIT    -0.285540  0.047885 -5.963060  2.475582e-09      0.047956   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            3.139955  1.689740e-03  
BETA_IVT_3_WAIT     2.025540  4.281197e-02  
BETA_IVT_4_NOW     -2.025540  4.281197e-02  
BETA_WT_WAIT       -5.954204  2.613414e-09  


# Model 10a - experiment no. 2, time-critical trips only, 2nd-dep utility: 
### dep1 = [ASC_now] 
### dep2 = [ASC_wait] + ivt_3 * [B_ivt4-3_wait] + wt * [B_WT_wait]

In [90]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_3_wait = Beta('BETA_IVT_4_3_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
WAIT_IVT_3 = wait_ivt_3
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW 

V2 = ASC_WAIT + \
     B_IVT_4_3_wait * WAIT_IVT_3 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [91]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_10a_EDI_timecrit_trips_ASC_scenario_2_2nd_dep_only"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	1.25
BETA_IVT_4_3_WAIT=	0.0181
BETA_WT_WAIT=	-0.286
                      Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT           1.251090  0.401908  3.112880  1.852713e-03      0.398442   
BETA_IVT_4_3_WAIT  0.018098  0.009115  1.985531  4.708542e-02      0.008935   
BETA_WT_WAIT      -0.285539  0.047885 -5.963049  2.475735e-09      0.047956   

                   Rob. t-test  Rob. p-value  
ASC_WAIT              3.139959  1.689713e-03  
BETA_IVT_4_3_WAIT     2.025521  4.281389e-02  
BETA_WT_WAIT         -5.954198  2.613498e-09  


# Model 11 - experiment no. 3, non-time-critical trips only: 
### dep1 = [ASC_now] + ivt_4 * [B_ivt4_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [92]:
# filter records
df_used = df[(df['time_crit']==0) & ((df['scenario'] == 5) | (df['scenario'] == 6))]
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
28,5,5,0,20,0,0,20,10,1
29,5,6,0,20,0,0,20,5,1
40,7,5,0,10,0,0,10,10,1
41,7,6,0,10,0,0,10,5,1
58,10,5,0,25,0,0,25,10,1
59,10,6,0,25,0,0,25,5,1
64,11,5,0,17,0,0,17,10,0
65,11,6,0,17,0,0,17,5,1
70,12,5,0,5,0,0,5,10,1


In [93]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [94]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [95]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_11_EDI_nontimecrit_trips_ASC_scenario_3"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	2.4
BETA_IVT_2_WAIT=	0.0207
BETA_IVT_4_NOW=	-0.0207
BETA_WT_WAIT=	-0.282
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         2.402979  0.476982  5.037883  4.707080e-07      0.528097   
BETA_IVT_2_WAIT  0.020739  0.006199  3.345541  8.212229e-04      0.008331   
BETA_IVT_4_NOW  -0.020739  0.006199 -3.345541  8.212228e-04      0.008331   
BETA_WT_WAIT    -0.281743  0.049909 -5.645186  1.650030e-08      0.048936   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            4.550259  5.357984e-06  
BETA_IVT_2_WAIT     2.489383  1.279649e-02  
BETA_IVT_4_NOW     -2.489383  1.279649e-02  
BETA_WT_WAIT       -5.757391  8.542399e-09  


# Model 11a - experiment no. 3, non-time-critical trips only, 2nd-dep utility: 
### dep1 = [ASC_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt4-2_wait] + wt * [B_WT_wait]

In [96]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_2_wait = Beta('BETA_IVT_4_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW 

V2 = ASC_WAIT + \
     B_IVT_4_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [97]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_11a_EDI_nontimecrit_trips_ASC_scenario_3_2nd_dep_only"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	2.4
BETA_IVT_4_2_WAIT=	0.0415
BETA_WT_WAIT=	-0.282
                      Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT           2.402980  0.476982  5.037886  4.707022e-07      0.528097   
BETA_IVT_4_2_WAIT  0.041477  0.012398  3.345549  8.211970e-04      0.016662   
BETA_WT_WAIT      -0.281744  0.049909 -5.645195  1.649938e-08      0.048936   

                   Rob. t-test  Rob. p-value  
ASC_WAIT              4.550261  5.357946e-06  
BETA_IVT_4_2_WAIT     2.489389  1.279630e-02  
BETA_WT_WAIT         -5.757399  8.541977e-09  


# Model 12 - experiment no. 3, time-critical trips only: 
### dep1 = [ASC_now] + ivt_4 * [B_ivt4_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt2_wait] + wt * [B_WT_wait]

In [98]:
# filter records
df_used = df[(df['time_crit']==1) & ((df['scenario'] == 5) | (df['scenario'] == 6))]
df_used.head(10)

,pass_ID,scenario,time_crit,now_ivt_4,now_ivt_3,wait_ivt_3,wait_ivt_2,WT_time,output_choice
index,,,,,,,,,
4,1,5,1,20,0,0,20,10,0
5,1,6,1,20,0,0,20,5,1
10,2,5,1,15,0,0,15,10,0
11,2,6,1,15,0,0,15,5,0
16,3,5,1,15,0,0,15,10,0
17,3,6,1,15,0,0,15,5,1
22,4,5,1,30,0,0,30,10,0
23,4,6,1,30,0,0,30,5,1
34,6,5,1,25,0,0,25,10,1


In [99]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_used)

In [100]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_now = Beta('BETA_IVT_4_NOW',0,None,None,0)
B_IVT_2_wait = Beta('BETA_IVT_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
NOW_IVT_4 = now_ivt_4
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW + \
     B_IVT_4_now * NOW_IVT_4

V2 = ASC_WAIT + \
     B_IVT_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [101]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_12_EDI_timecrit_trips_ASC_scenario_3"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	1.37
BETA_IVT_2_WAIT=	0.011
BETA_IVT_4_NOW=	-0.011
BETA_WT_WAIT=	-0.307
                    Value   Std err    t-test       p-value  Rob. Std err  \
ASC_WAIT         1.370346  0.402308  3.406208  6.587188e-04      0.399427   
BETA_IVT_2_WAIT  0.010963  0.004636  2.364894  1.803524e-02      0.004603   
BETA_IVT_4_NOW  -0.010963  0.004636 -2.364894  1.803524e-02      0.004603   
BETA_WT_WAIT    -0.306627  0.048233 -6.357230  2.054243e-10      0.048342   

                 Rob. t-test  Rob. p-value  
ASC_WAIT            3.430780  6.018474e-04  
BETA_IVT_2_WAIT     2.381512  1.724171e-02  
BETA_IVT_4_NOW     -2.381512  1.724171e-02  
BETA_WT_WAIT       -6.342859  2.255394e-10  


# Model 12a - experiment no. 3, time-critical trips only, 2nd-dep utility: 
### dep1 = [ASC_now] 
### dep2 = [ASC_wait] + ivt_2 * [B_ivt4-2_wait] + wt * [B_WT_wait]

In [102]:
# coefficients
ASC_NOW = Beta('ASC_NOW',0,None,None,1)
ASC_WAIT = Beta('ASC_WAIT',0,None,None,0)
B_IVT_4_2_wait = Beta('BETA_IVT_4_2_WAIT',0,None,None,0)
B_WT_wait = Beta('BETA_WT_WAIT',0,None,None,0)

# variables
WAIT_IVT_2 = wait_ivt_2
WAIT_TIME = WT_time

# utilities
V1 = ASC_NOW 

V2 = ASC_WAIT + \
     B_IVT_4_2_wait * WAIT_IVT_2 + \
     B_WT_wait * WAIT_TIME

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [103]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_12a_EDI_timecrit_trips_ASC_scenario_3_2nd_dep_only"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_WAIT=	1.37
BETA_IVT_4_2_WAIT=	0.0219
BETA_WT_WAIT=	-0.307
                      Value   Std err   t-test       p-value  Rob. Std err  \
ASC_WAIT           1.370346  0.402308  3.40621  6.587140e-04      0.399427   
BETA_IVT_4_2_WAIT  0.021926  0.009271  2.36491  1.803448e-02      0.009207   
BETA_WT_WAIT      -0.306628  0.048233 -6.35724  2.054115e-10      0.048342   

                   Rob. t-test  Rob. p-value  
ASC_WAIT              3.430782  6.018438e-04  
BETA_IVT_4_2_WAIT     2.381526  1.724109e-02  
BETA_WT_WAIT         -6.342866  2.255294e-10  


In [ ]:
########################################
#
# @file 01logit.py
# @author: Michel Bierlaire, EPFL
# @date: Thu Sep  6 15:14:39 2018
#
# Logit model
# Three alternatives: Train, Car and Swissmetro
# SP data
#
#######################################

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

# The Pandas data structure is available as database.data. Use all the
# Pandas functions to invesigate the database
#print(database.data.describe())

from headers import *

# Removing some observations can be done directly using pandas.
#remove = (((database.data.PURPOSE != 1) & (database.data.PURPOSE != 3)) | (database.data.CHOICE == 0))
#database.data.drop(database.data[remove].index,inplace=True)

# Here we use the "biogeme" way for backward compatibility
exclude = (( PURPOSE != 1 ) * (  PURPOSE   !=  3  ) +  ( CHOICE == 0 )) > 0
database.remove(exclude)


ASC_CAR = Beta('ASC_CAR',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_SM = Beta('ASC_SM',0,None,None,1)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)

SM_COST =  SM_CO   * (  GA   ==  0  ) 
TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)

TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100,database)

V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "01logit"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
